In [ ]:
import pandas as pd

# ----------------------------------
# 0. 파일 경로 설정
# ----------------------------------
cohort_path = "./cohort/cohort_ver41_remove_old_col.csv"
d_labitems_path = "../../data/MIMIC4-hosp-icu/d_labitems.csv"
labevents_path = "../../data/MIMIC4-hosp-icu/labevents.csv"
output_path = "./cohort/cohort_ver41_with_troponin.csv"

# ----------------------------------
# 1. Cohort 불러오기
# ----------------------------------
cohort = pd.read_csv(cohort_path)

if 'hadm_id' not in cohort.columns:
    raise ValueError("cohort에 'hadm_id' 컬럼이 없습니다. join key를 확인해주세요.")

cohort['hadm_id'] = cohort['hadm_id'].astype('Int64')

# ----------------------------------
# 2. Troponin itemid 자동 탐색 (d_labitems 사용)
#    label에 'TROPONIN' 이 포함된 항목을 Troponin으로 간주
# ----------------------------------
d_lab = pd.read_csv(d_labitems_path)

troponin_mask = d_lab['label'].str.contains('TROPONIN', case=False, na=False)
troponin_itemids = d_lab.loc[troponin_mask, 'itemid'].unique().tolist()

print(f"[INFO] Troponin으로 인식된 itemid 개수: {len(troponin_itemids)}")
print(f"[INFO] Troponin itemid 리스트 (일부): {troponin_itemids[:20]}")

if len(troponin_itemids) == 0:
    raise ValueError("TROPONIN 이 포함된 lab label을 찾지 못했습니다. d_labitems를 확인해주세요.")

# ----------------------------------
# 3. labevents를 chunk로 읽으면서 Troponin만 수집
#    (파일이 너무 크기 때문에 전체를 한 번에 읽지 않도록)
# ----------------------------------
usecols = [
    "labevent_id",
    "subject_id",
    "hadm_id",
    "itemid",
    "charttime",
    "storetime",
    "value",
    "valuenum",
    "flag",
]

troponin_rows = []

chunksize = 1_000_000  # 메모리 상황에 따라 조정

for chunk in pd.read_csv(
    labevents_path,
    usecols=usecols,
    chunksize=chunksize,
    parse_dates=["charttime", "storetime"],
):
    # hadm_id 없는 행은 일단 제거 (cohort join 어려움)
    chunk = chunk[chunk["hadm_id"].notna()]
    chunk["hadm_id"] = chunk["hadm_id"].astype("Int64")

    # Troponin itemid만 필터링
    chunk_tp = chunk[chunk["itemid"].isin(troponin_itemids)]

    if len(chunk_tp) > 0:
        troponin_rows.append(chunk_tp)

if len(troponin_rows) == 0:
    raise ValueError("labevents에서 Troponin 행을 찾지 못했습니다. itemid 및 경로를 확인해주세요.")

troponin = pd.concat(troponin_rows, ignore_index=True)
print(f"[INFO] Troponin 행 개수: {len(troponin)}")

# ----------------------------------
# 4. Troponin 파생 변수 계산
#    - first_troponin_charttime          : hadm_id 별 첫 charttime
#    - troponin_end_charttime            : hadm_id 별 마지막 storetime
#    - first_troponin_positive_charttime : flag == 'abnormal' 인 첫 charttime
#    - troponin_count                    : Troponin 검사 횟수
# ----------------------------------

# 4-1. 첫 Troponin 검사 시간
first_trop = (
    troponin.groupby("hadm_id")["charttime"]
    .min()
    .reset_index()
    .rename(columns={"charttime": "first_troponin_charttime"})
)

# 4-2. Troponin 검사 종료시간 (storetime 중 최댓값)
trop_end = (
    troponin.groupby("hadm_id")["storetime"]
    .max()
    .reset_index()
    .rename(columns={"storetime": "troponin_end_charttime"})
)

# 4-3. 첫 양성 Troponin 검사 시간
#     기준: flag == 'abnormal'
#     (필요하면 valuenum 기준 threshold로 나중에 바꿀 수 있음)
tp_pos = troponin[troponin["flag"].str.lower() == "abnormal"].copy()
first_pos = (
    tp_pos.groupby("hadm_id")["charttime"]
    .min()
    .reset_index()
    .rename(columns={"charttime": "first_troponin_positive_charttime"})
)

# 4-4. Troponin 검사 횟수
trop_count = (
    troponin.groupby("hadm_id")["labevent_id"]
    .count()
    .reset_index()
    .rename(columns={"labevent_id": "troponin_count"})
)

# ----------------------------------
# 5. Cohort에 merge
# ----------------------------------
cohort = cohort.merge(first_trop, on="hadm_id", how="left")
cohort = cohort.merge(trop_end, on="hadm_id", how="left")
cohort = cohort.merge(first_pos, on="hadm_id", how="left")
cohort = cohort.merge(trop_count, on="hadm_id", how="left")

# ----------------------------------
# 6. 저장
# ----------------------------------
cohort.to_csv(output_path, index=False)
print(f"[DONE] Troponin 변수 4개를 추가한 cohort 저장 완료: {output_path}")
